In [4]:
%run /home/ptenkaate/scratch/Master-Thesis/convert_ipynb_to_py_files.ipynb

[NbConvertApp] Converting notebook /home/ptenkaate/scratch/Master-Thesis/data_classes/new_dataset.ipynb to script
[NbConvertApp] Writing 4177 bytes to py_files/new_dataset.py
[NbConvertApp] Converting notebook /home/ptenkaate/scratch/Master-Thesis/model_classes/cnn_model.ipynb to script
[NbConvertApp] Writing 6584 bytes to py_files/cnn_model.py
[NbConvertApp] Converting notebook /home/ptenkaate/scratch/Master-Thesis/model_classes/pigan_model.ipynb to script
[NbConvertApp] Writing 3587 bytes to py_files/pigan_model.py
[NbConvertApp] Converting notebook /home/ptenkaate/scratch/Master-Thesis/pi-gan/pi_gan_functions.ipynb to script
[NbConvertApp] Writing 21735 bytes to py_files/pi_gan_functions.py
Saved py files.


In [5]:
import torch
import torch.nn.functional as F

from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import Resize, Compose, ToTensor, Normalize

import argparse
import os
import math 
import skimage
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

import time
import pickle

from datetime import datetime
from pathlib import Path

from py_files.new_dataset import *

from py_files.cnn_model import *
from py_files.pigan_model import *

from py_files.pi_gan_functions import *

Loaded all helper functions.


#### Train the model

In [3]:
def train():  
    
    ##### path to wich the model should be saved #####
    path = get_folder(ARGS)
    
    ##### save ARGS #####
    with open(f"{path}/ARGS.txt", "w") as f:
        print(vars(ARGS), file=f)
        
    ##### data preparation #####
    train_dl, val_dl, test_dl = initialize_dataloaders(ARGS) 
    
            
    ##### initialize models and optimizers #####
    models, optims, schedulers = load_models_and_optims(ARGS)

    ##### load pretrained model #####
    if ARGS.pretrained: 
        print(f"Loading pretrained model from '{ARGS.pretrained}'.")
        load_models(ARGS.pretrained, ARGS.pretrained_best, 
                    models, optims)
    
    ##### loss function #####
    criterions = [nn.BCELoss(), nn.MSELoss()]
#     criterions = [nn.BCELoss(), nn.L1Loss()]
        
    ##### epoch, train loss mean, train loss std, #####
    ##### val loss mean, val loss std #####
    mask_losses = np.empty((0, 5))
    pcmra_losses = np.empty((0, 5))
    dice_losses = np.empty((0, 5))

    batch_count = 0     
    
    for ep in range(ARGS.epochs):
    
        t = time.time() 

        for model in models.values():
            model.train()

        t_loss_mean, t_loss_std, \
        t_p_loss_mean, t_p_loss_std, \
        batch_count = train_epoch(train_dl, models, optims, schedulers,
                                  criterions, batch_count, ARGS)
        
        print(f"Epoch {ep}, train loss: {t_loss_mean}, train pcmra loss: {t_p_loss_mean}")
        
        if ep % ARGS.eval_every == 0: 

            print(f"Epoch {ep} took {round(time.time() - t, 2)} seconds.")
            
            t_mask_mean, t_mask_std, \
            t_pcmra_mean, t_pcmra_std, \
            t_dice_mean, t_dice_std = val_epoch(train_dl, models, criterions, ARGS)
            
            v_mask_mean, v_mask_std, \
            v_pcmra_mean, v_pcmra_std, \
            v_dice_mean, v_dice_std = val_epoch(val_dl, models, criterions, ARGS)
            
            mask_losses = np.append(mask_losses, [[ep ,t_mask_mean, t_mask_std, 
                                         v_mask_mean, v_mask_std]], axis=0)
            
            pcmra_losses = np.append(pcmra_losses, [[ep ,t_pcmra_mean, t_pcmra_std, 
                                         v_pcmra_mean, v_pcmra_std]], axis=0)
            
            dice_losses = np.append(dice_losses, [[ep ,t_dice_mean, t_dice_std, 
                                         v_dice_mean, v_dice_std]], axis=0)
            
            save_info(path, mask_losses, pcmra_losses, dice_losses, models, optims, save_last=True)

## Run as .ipynb

In [4]:
torch.cuda.empty_cache()    

ARGS = init_ARGS()

ARGS.n_coords_sample, ARGS.batch_size = -1, 2
ARGS.pcmra_lambda = 10
ARGS.epochs = 70
ARGS.print_models = False
ARGS.rotate, ARGS.translate, ARGS.flip = True, True, False


ARGS.scheduler_on = "combined"
ARGS.reconstruction = "pcmra"

ARGS.share_mapping = False

ARGS.cnn_setup = 1
ARGS.mapping_setup = 2

print(vars(ARGS))

train()  

torch.cuda.empty_cache()    


{'device': 'GPU', 'print_models': False, 'name': '', 'pretrained': None, 'pretrained_best': 'train', 'reconstruction': 'pcmra', 'share_mapping': False, 'pcmra_lambda': 5, 'mask_lambda': 1.0, 'dataset': 'small', 'rotate': True, 'translate': True, 'flip': False, 'norm_min_max': [0, 1], 'seed': 34, 'epochs': 70, 'batch_size': 8, 'eval_every': 5, 'shuffle': True, 'n_coords_sample': 20000, 'cnn_setup': 1, 'mapping_setup': 2, 'dim_hidden': 256, 'siren_hidden_layers': 3, 'first_omega_0': 30.0, 'hidden_omega_0': 30.0, 'pcmra_first_omega_0': 30.0, 'pcmra_hidden_omega_0': 30.0, 'cnn_lr': 0.0001, 'cnn_wd': 0, 'mapping_lr': 0.0001, 'pcmra_mapping_lr': 0.0001, 'siren_lr': 0.0001, 'siren_wd': 0, 'pcmra_siren_lr': 0.0001, 'pcmra_siren_wd': 0, 'scheduler_on': 'combined'}
----------------------------------
Using device for training: cuda
----------------------------------
Train subjects: 1596
[['16-01-22_Jarik_kt-pca_done2', 'Aorta Volunteers', 'scaled_rot 0 (-, -)'], ['16-01-27 Valentine kt-pca_done2'

Epoch 59, train loss: 0.019759, train pcmra loss: 0.001038
Epoch 60, train loss: 0.018794, train pcmra loss: 0.00097
Epoch 60 took 59.6 seconds.
Train mask loss: 	 0.01728,     pcmra loss: 	 0.00099,     	dice loss: 	 0.12812.
Eval  mask loss: 	 0.06702,     pcmra loss: 	 0.00232,     	dice loss: 	 0.31964.
New best train loss, saving model.
Epoch 61, train loss: 0.019628, train pcmra loss: 0.001049
Epoch 62, train loss: 0.019354, train pcmra loss: 0.001028
Epoch 63, train loss: 0.019499, train pcmra loss: 0.001021
Epoch 64, train loss: 0.018803, train pcmra loss: 0.001011
Epoch 65, train loss: 0.018929, train pcmra loss: 0.000973
Epoch 65 took 59.52 seconds.
Train mask loss: 	 0.01776,     pcmra loss: 	 0.00094,     	dice loss: 	 0.13507.
Eval  mask loss: 	 0.06727,     pcmra loss: 	 0.00236,     	dice loss: 	 0.31996.
New best train loss, saving model.
Epoch 66, train loss: 0.018333, train pcmra loss: 0.000979
Epoch 67, train loss: 0.018274, train pcmra loss: 0.000976
Epoch 68, train

Epoch 49, train loss: 0.021031, train pcmra loss: 0.001122
Epoch 50, train loss: 0.01992, train pcmra loss: 0.001061
Epoch 50 took 60.35 seconds.
Train mask loss: 	 0.01958,     pcmra loss: 	 0.00101,     	dice loss: 	 0.14972.
Eval  mask loss: 	 0.06692,     pcmra loss: 	 0.00229,     	dice loss: 	 0.33619.
New best train loss, saving model.
Epoch 51, train loss: 0.02042, train pcmra loss: 0.001082
Epoch 52, train loss: 0.02001, train pcmra loss: 0.001057
Epoch 53, train loss: 0.020023, train pcmra loss: 0.001043
Epoch 54, train loss: 0.020363, train pcmra loss: 0.001049
Epoch 55, train loss: 0.019708, train pcmra loss: 0.001048
Epoch 55 took 60.67 seconds.
Train mask loss: 	 0.02166,     pcmra loss: 	 0.00099,     	dice loss: 	 0.16818.
Eval  mask loss: 	 0.06929,     pcmra loss: 	 0.00242,     	dice loss: 	 0.34777.
New best train loss, saving model.
Epoch 56, train loss: 0.019517, train pcmra loss: 0.001017
Epoch 57, train loss: 0.019659, train pcmra loss: 0.001032
Epoch 58, train 

Epoch 39, train loss: 0.026106, train pcmra loss: 0.001367
Epoch 40, train loss: 0.02588, train pcmra loss: 0.00134
Epoch 40 took 40.73 seconds.
Train mask loss: 	 0.02495,     pcmra loss: 	 0.00144,     	dice loss: 	 0.18983.
Eval  mask loss: 	 0.0631,     pcmra loss: 	 0.00218,     	dice loss: 	 0.35214.
New best train loss, saving model.
Epoch 41, train loss: 0.025245, train pcmra loss: 0.00135
Epoch 42, train loss: 0.024614, train pcmra loss: 0.001322
Epoch 43, train loss: 0.025127, train pcmra loss: 0.001332
Epoch 44, train loss: 0.025862, train pcmra loss: 0.00138
Epoch 45, train loss: 0.025018, train pcmra loss: 0.001348
Epoch 45 took 40.92 seconds.
Train mask loss: 	 0.02492,     pcmra loss: 	 0.00125,     	dice loss: 	 0.18463.
Eval  mask loss: 	 0.06198,     pcmra loss: 	 0.00214,     	dice loss: 	 0.34358.
New best train loss, saving model.
Epoch 46, train loss: 0.024255, train pcmra loss: 0.001279
Epoch 47, train loss: 0.023569, train pcmra loss: 0.001218
Epoch 48, train lo

Epoch 29, train loss: 0.030631, train pcmra loss: 0.001563
Epoch 30, train loss: 0.029297, train pcmra loss: 0.00154
Epoch 30 took 40.65 seconds.
Train mask loss: 	 0.02824,     pcmra loss: 	 0.00157,     	dice loss: 	 0.2172.
Eval  mask loss: 	 0.05789,     pcmra loss: 	 0.00203,     	dice loss: 	 0.36193.
New best train loss, saving model.
Epoch 31, train loss: 0.028544, train pcmra loss: 0.001506
Epoch 32, train loss: 0.029808, train pcmra loss: 0.001602
Epoch 33, train loss: 0.027441, train pcmra loss: 0.001497
Epoch 34, train loss: 0.027723, train pcmra loss: 0.001467
Epoch 35, train loss: 0.027129, train pcmra loss: 0.001437
Epoch 35 took 40.33 seconds.
Train mask loss: 	 0.02864,     pcmra loss: 	 0.00158,     	dice loss: 	 0.21852.
Eval  mask loss: 	 0.06117,     pcmra loss: 	 0.00205,     	dice loss: 	 0.34503.
New best train loss, saving model.
Epoch 36, train loss: 0.028665, train pcmra loss: 0.001501
Epoch 37, train loss: 0.026064, train pcmra loss: 0.001395
Epoch 38, train

In [5]:
torch.cuda.empty_cache()    

# ARGS.n_coords_sample=-1
# ARGS.batch_size = 4

versions = [(20000, 8), (5000, 24)]

ARGS = init_ARGS()


for ARGS.n_coords_sample, ARGS.batch_size in versions: 
    for ARGS.pcmra_lambda in [5, 10]:
        ARGS.epochs = 70
        ARGS.print_models = False
        ARGS.rotate, ARGS.translate, ARGS.flip = True, True, False


        ARGS.scheduler_on = "combined"
        ARGS.reconstruction = "mask"

        ARGS.share_mapping = False

        ARGS.cnn_setup = 1
        ARGS.mapping_setup = 2

        print(vars(ARGS))

        train()  

        torch.cuda.empty_cache()    

{'device': 'GPU', 'print_models': False, 'name': '', 'pretrained': None, 'pretrained_best': 'train', 'reconstruction': 'mask', 'share_mapping': False, 'pcmra_lambda': 5, 'mask_lambda': 1.0, 'dataset': 'small', 'rotate': True, 'translate': True, 'flip': False, 'norm_min_max': [0, 1], 'seed': 34, 'epochs': 70, 'batch_size': 8, 'eval_every': 5, 'shuffle': True, 'n_coords_sample': 20000, 'cnn_setup': 1, 'mapping_setup': 2, 'dim_hidden': 256, 'siren_hidden_layers': 3, 'first_omega_0': 30.0, 'hidden_omega_0': 30.0, 'pcmra_first_omega_0': 30.0, 'pcmra_hidden_omega_0': 30.0, 'cnn_lr': 0.0001, 'cnn_wd': 0, 'mapping_lr': 0.0001, 'pcmra_mapping_lr': 0.0001, 'siren_lr': 0.0001, 'siren_wd': 0, 'pcmra_siren_lr': 0.0001, 'pcmra_siren_wd': 0, 'scheduler_on': 'combined'}
----------------------------------
Using device for training: cuda
----------------------------------
Train subjects: 1596
[['16-01-22_Jarik_kt-pca_done2', 'Aorta Volunteers', 'scaled_rot 0 (-, -)'], ['16-01-27 Valentine kt-pca_done2',

Epoch 64, train loss: 0.017906, train pcmra loss: 0.0
Epoch 65, train loss: 0.017639, train pcmra loss: 0.0
Epoch 65 took 42.99 seconds.
Train mask loss: 	 0.01908,     pcmra loss: 	 0.0,     	dice loss: 	 0.14696.
Eval  mask loss: 	 0.05252,     pcmra loss: 	 0.0,     	dice loss: 	 0.27383.
New best train loss, saving model.
Epoch 66, train loss: 0.017902, train pcmra loss: 0.0
Epoch 67, train loss: 0.01788, train pcmra loss: 0.0
Epoch 68, train loss: 0.017267, train pcmra loss: 0.0
Epoch 69, train loss: 0.01692, train pcmra loss: 0.0
{'device': 'GPU', 'print_models': False, 'name': '', 'pretrained': None, 'pretrained_best': 'train', 'reconstruction': 'mask', 'share_mapping': False, 'pcmra_lambda': 10, 'mask_lambda': 1.0, 'dataset': 'small', 'rotate': True, 'translate': True, 'flip': False, 'norm_min_max': [0, 1], 'seed': 34, 'epochs': 70, 'batch_size': 8, 'eval_every': 5, 'shuffle': True, 'n_coords_sample': 20000, 'cnn_setup': 1, 'mapping_setup': 2, 'dim_hidden': 256, 'siren_hidden_l

Epoch 57, train loss: 0.019064, train pcmra loss: 0.0
Epoch 58, train loss: 0.018184, train pcmra loss: 0.0
Epoch 59, train loss: 0.018307, train pcmra loss: 0.0
Epoch 60, train loss: 0.018685, train pcmra loss: 0.0
Epoch 60 took 42.38 seconds.
Train mask loss: 	 0.01808,     pcmra loss: 	 0.0,     	dice loss: 	 0.13586.
Eval  mask loss: 	 0.05307,     pcmra loss: 	 0.0,     	dice loss: 	 0.27466.
New best train loss, saving model.
Epoch 61, train loss: 0.018207, train pcmra loss: 0.0
Epoch 62, train loss: 0.017931, train pcmra loss: 0.0
Epoch 63, train loss: 0.017468, train pcmra loss: 0.0
Epoch 64, train loss: 0.017955, train pcmra loss: 0.0
Epoch 65, train loss: 0.017484, train pcmra loss: 0.0
Epoch 65 took 42.4 seconds.
Train mask loss: 	 0.01836,     pcmra loss: 	 0.0,     	dice loss: 	 0.13553.
Eval  mask loss: 	 0.05216,     pcmra loss: 	 0.0,     	dice loss: 	 0.26113.
New best train loss, saving model.
Epoch 66, train loss: 0.018075, train pcmra loss: 0.0
Epoch 67, train loss:

Epoch 51, train loss: 0.02108, train pcmra loss: 0.0
Epoch 52, train loss: 0.021521, train pcmra loss: 0.0
Epoch 53, train loss: 0.020259, train pcmra loss: 0.0
Epoch 54, train loss: 0.02083, train pcmra loss: 0.0
Epoch 55, train loss: 0.021342, train pcmra loss: 0.0
Epoch 55 took 35.96 seconds.
Train mask loss: 	 0.02151,     pcmra loss: 	 0.0,     	dice loss: 	 0.16457.
Eval  mask loss: 	 0.04651,     pcmra loss: 	 0.0,     	dice loss: 	 0.27606.
New best train loss, saving model.
Epoch 56, train loss: 0.021353, train pcmra loss: 0.0
Epoch 57, train loss: 0.020227, train pcmra loss: 0.0
Epoch 58, train loss: 0.019823, train pcmra loss: 0.0
Epoch 59, train loss: 0.02015, train pcmra loss: 0.0
Epoch 60, train loss: 0.020381, train pcmra loss: 0.0
Epoch 60 took 36.35 seconds.
Train mask loss: 	 0.0219,     pcmra loss: 	 0.0,     	dice loss: 	 0.16924.
Eval  mask loss: 	 0.04392,     pcmra loss: 	 0.0,     	dice loss: 	 0.25271.
New best train loss, saving model.
Epoch 61, train loss: 0.

Epoch 46, train loss: 0.022817, train pcmra loss: 0.0
Epoch 47, train loss: 0.022304, train pcmra loss: 0.0
Epoch 48, train loss: 0.022241, train pcmra loss: 0.0
Epoch 49, train loss: 0.022533, train pcmra loss: 0.0
Epoch 50, train loss: 0.022207, train pcmra loss: 0.0
Epoch 50 took 36.39 seconds.
Train mask loss: 	 0.02283,     pcmra loss: 	 0.0,     	dice loss: 	 0.17448.
Eval  mask loss: 	 0.04405,     pcmra loss: 	 0.0,     	dice loss: 	 0.26968.
New best train loss, saving model.
Epoch 51, train loss: 0.022177, train pcmra loss: 0.0
Epoch 52, train loss: 0.021445, train pcmra loss: 0.0
Epoch 53, train loss: 0.021176, train pcmra loss: 0.0
Epoch 54, train loss: 0.022031, train pcmra loss: 0.0
Epoch 55, train loss: 0.021044, train pcmra loss: 0.0
Epoch 55 took 36.46 seconds.
Train mask loss: 	 0.02052,     pcmra loss: 	 0.0,     	dice loss: 	 0.15709.
Eval  mask loss: 	 0.04441,     pcmra loss: 	 0.0,     	dice loss: 	 0.26368.
New best train loss, saving model.
Epoch 56, train loss

In [6]:
# ARGS = init_ARGS()

# ARGS.print_models = False
# ARGS.epochs = 250
# ARGS.eval_every = 20

# cnn_mapping_combis = [[1, 1], [1, 2], [2, 3], [2, 4]]
# transformations = [(False, False, False)]
# share_mapping = [False, True]
# reconstruction = ["pcmra", "both", "mask"]
# lambdas = [(1, 1), (1, 10)]
# omega_0s = [5, 30, 100]

# for ARGS.cnn_setup, ARGS.mapping_setup in cnn_mapping_combis: 
#     for ARGS.rotate, ARGS.translate, ARGS.flip in transformations: 
#         for ARGS.share_mapping in share_mapping: 
#             for ARGS.reconstruction in reconstruction: 
#                 for ARGS.mask_lambda, ARGS.pcmra_lambda in lambdas: 
#                     for ARGS.first_omega_0 in omega_0s: 
#                         print(vars(ARGS))
#                         train()  
                    


In [7]:
# ARGS = init_ARGS()

# ARGS.print_models = False
# ARGS.epochs = 51

# transformations = [(True, True, False), (True, True, True)]
# cnn_mapping_combis = [[1, 2], [2, 3], [2, 4]]
# share_mapping = [False, True]
# reconstruction = ["pcmra", "both", "mask"]
# lambdas = [(1, 1), (1, 10)]
# omega_0s = [30]

# for ARGS.rotate, ARGS.translate, ARGS.flip in transformations: 
#     for ARGS.cnn_setup, ARGS.mapping_setup in cnn_mapping_combis: 
#         for ARGS.share_mapping in share_mapping: 
#             for ARGS.reconstruction in reconstruction: 
#                 for ARGS.mask_lambda, ARGS.pcmra_lambda in lambdas: 
#                     for ARGS.first_omega_0 in omega_0s: 
#                         print(vars(ARGS))
#                         train()  
                    


## Run as .py

In [8]:
if __name__ == "__main__":
    PARSER = argparse.ArgumentParser()

    # Arguments for training
    PARSER.add_argument('--device', type=str, default="GPU", 
                        help='Device that should be used.')

    PARSER.add_argument('--print_models', type=str, default="GPU", 
                        help='Print the models after initialization or not.')

    PARSER.add_argument('--name', type=str, default="", 
                        help='Name of the folder where the output should be saved.')
    
    PARSER.add_argument('--pretrained', type=str, default=None, 
                        help='Folder name of pretrained model that should be loaded.')
    
    PARSER.add_argument('--pretrained_best', type=str, default="train", 
                        help='Pretrained model with lowest [train, val] loss.')
    
    
    # PCMRA and Mask reconstruction
    PARSER.add_argument('--reconstruction', type=str, default="pcmra", 
                        help='Do we want the CNN trained on reconstruction of pcmra/mask/both.')
    
    PARSER.add_argument('--share_mapping', type=bool, default=False, 
                        help='Do we want to share the mapping from latent repr to gamma and beta?')
    
    PARSER.add_argument('--pcmra_lambda', type=float, default=1., 
                        help='Multiplier for pcmra reconstruction loss')
    
    PARSER.add_argument('--mask_lambda', type=float, default=1., 
                        help='Multiplier for mask reconstruction loss')
    
    
    # data
    PARSER.add_argument('--dataset', type=str, default="small", 
                        help='The dataset which we train on.')
    
    PARSER.add_argument('--rotate', type=bool, default=True, 
                        help='Rotations of the same image')
    
    PARSER.add_argument('--translate', type=bool, default=True, 
                        help='Translations of the same image')
    
    PARSER.add_argument('--flip', type=bool, default=True, 
                        help='Flips of the same image')
    
    PARSER.add_argument('--norm_min_max', type=list, default=[0, 1], 
                        help='List with min and max for normalizing input.')
    
    PARSER.add_argument('--seed', type=int, default=34, 
                        help='Seed for initializig dataloader')
    
    
    # train variables
    PARSER.add_argument('--epochs', type=int, default=51, 
                        help='Number of epochs.')
    
    PARSER.add_argument('--batch_size', type=int, default=24, 
                        help='Number of epochs.')
        
    PARSER.add_argument('--eval_every', type=int, default=5, 
                        help='Set the # epochs after which evaluation should be done.')
    
    PARSER.add_argument('--shuffle', type=bool, default=True, 
                        help='Shuffle the train dataloader?')
    
    PARSER.add_argument('--n_coords_sample', type=int, default=5000, 
                        help='Number of coordinates that should be sampled for each subject.')
    
    
    # CNN
    PARSER.add_argument('--cnn_setup', type=int, default=1, 
                        help='Setup of the CNN.')

    
    # Mapping
    PARSER.add_argument('--mapping_setup', type=int, default=1, 
                        help='Setup of the Mapping network.')

    
    # SIREN
    PARSER.add_argument('--dim_hidden', type=int, default=256, 
                        help='Dimension of hidden SIREN layers.')
    
    PARSER.add_argument('--siren_hidden_layers', type=int, default=3, 
                        help='Number of hidden SIREN layers.')
    
    
    PARSER.add_argument('--first_omega_0', type=float, default=30., 
                        help='Omega_0 of first layer.')
    
    PARSER.add_argument('--hidden_omega_0', type=float, default=30., 
                        help='Omega_0 of hidden layer.')
    
    
    PARSER.add_argument('--pcmra_first_omega_0', type=float, default=30., 
                        help='Omega_0 of first layer of PCMRA siren.')
    
    PARSER.add_argument('--pcmra_hidden_omega_0', type=float, default=30., 
                        help='Omega_0 of hidden layer of PCMRA siren.')
    
    
    # optimizers
    PARSER.add_argument('--cnn_lr', type=float, default=1e-4, 
                        help='Learning rate of cnn optim.')

    PARSER.add_argument('--cnn_wd', type=float, default=0, 
                        help='Weight decay of cnn optim.')

    
    PARSER.add_argument('--mapping_lr', type=float, default=1e-4, 
                        help='Learning rate of siren optim.')
    
    PARSER.add_argument('--pcmra_mapping_lr', type=float, default=1e-4, 
                        help='Learning rate of siren optim.')
    

    PARSER.add_argument('--siren_lr', type=float, default=1e-4, 
                        help='Learning rate of siren optim.')

    PARSER.add_argument('--siren_wd', type=float, default=0, 
                        help='Weight decay of siren optim.')
    
    
    PARSER.add_argument('--pcmra_siren_lr', type=float, default=1e-4, 
                        help='Learning rate of PCMRA siren optim.')    
    
    PARSER.add_argument('--pcmra_siren_wd', type=float, default=0, 
                        help='Weight decay of PCMRA siren optim.')
    
    
    PARSER.add_argument('--scheduler_on', type=str, default="combined", 
                        help='Schedule lr on pcmra/mask/combined loss.')
    
 
    ARGS = PARSER.parse_args()
    
    train()

usage: ipykernel_launcher.py [-h] [--device DEVICE]
                             [--print_models PRINT_MODELS] [--name NAME]
                             [--pretrained PRETRAINED]
                             [--pretrained_best PRETRAINED_BEST]
                             [--reconstruction RECONSTRUCTION]
                             [--share_mapping SHARE_MAPPING]
                             [--pcmra_lambda PCMRA_LAMBDA]
                             [--mask_lambda MASK_LAMBDA] [--dataset DATASET]
                             [--rotate ROTATE] [--translate TRANSLATE]
                             [--flip FLIP] [--norm_min_max NORM_MIN_MAX]
                             [--seed SEED] [--epochs EPOCHS]
                             [--batch_size BATCH_SIZE]
                             [--eval_every EVAL_EVERY] [--shuffle SHUFFLE]
                             [--n_coords_sample N_COORDS_SAMPLE]
                             [--cnn_setup CNN_SETUP]
                             [--mapping_se

SystemExit: 2

/home/ptenkaate/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3351: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
